<a href="https://colab.research.google.com/github/BhavithavSetty/Liver-Disease-Diagnosis/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade scikit-learn xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.3
    Uninstalling xgboost-2.1.3:
      Successfully uninstalled xgboost-2.1.3


In [ ]:
import sklearn
import xgboost
import lightgbm
import pandas
import numpy
import joblib

print("scikit-learn version:", sklearn.__version__)
print("xgboost version:", xgboost.__version__)
print("lightgbm version:", lightgbm.__version__)
print("pandas version:", pandas.__version__)
print("numpy version:", numpy.__version__)
print("joblib version:", joblib.__version__)

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


scikit-learn version: 1.6.1
xgboost version: 2.1.4
lightgbm version: 4.5.0
pandas version: 2.2.2
numpy version: 1.26.4
joblib version: 1.4.2


In [ ]:
import pandas as pd
import numpy as np
file_path = '/content/rebalanced_dataset.csv'

# Load the file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

,Age of the patient,Gender of the patient,Total Bilirubin,Direct Bilirubin,Alkphos Alkaline Phosphotase,Sgpt Alamine Aminotransferase,Sgot Aspartate Aminotransferase,Total Protiens,ALB Albumin,A/G Ratio Albumin and Globulin Ratio,Result
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,0
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,0
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,0
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,0
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,0


In [ ]:
from sklearn.model_selection import train_test_split

# Separate features (X) and target (y)
# Replace 'target' with your target column name
X = df.drop('Result', axis=1)
y = df['Result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 4: Train base models
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Initialize base models
rf = RandomForestClassifier(random_state=42)
ada = AdaBoostClassifier(random_state=42)
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
lgbm = LGBMClassifier(random_state=42)

# Train base models
rf.fit(X_train, y_train)
ada.fit(X_train, y_train)
xgb.fit(X_train, y_train)
lgbm.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:48:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 11113, number of negative: 11056
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1764
[LightGBM] [Info] Number of data points in the train set: 22169, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501286 -> initscore=0.005142
[LightGBM] [Info] Start training from score 0.005142


LGBMClassifier(random_state=42)

In [ ]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("rebalanced_dataset.csv")  # Replace with your actual dataset file

# Randomly select 10,000 rows for training
train_df = df.sample(n=10000, random_state=42)

# Select remaining 31,000 rows for testing
test_df = df.drop(train_df.index)

# Save the datasets
train_df.to_csv("retraining.csv", index=False)
test_df.to_csv("retesting.csv", index=False)

print("Training and testing datasets saved successfully!")


Training and testing datasets saved successfully!


In [ ]:
# Step 4: Train and evaluate the Random Forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Initialize the Random Forest model
rf = RandomForestClassifier(
    random_state=42,
    max_depth=10,           # Limit tree depth
    min_samples_split=10,   # Minimum samples to split a node
    min_samples_leaf=5      # Minimum samples at a leaf node
)

# Train the model
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Print the evaluation metrics
print("Random Forest Model Evaluation:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

Random Forest Model Evaluation:
Accuracy: 0.9287
Precision: 0.8786
Recall: 0.9961
F1 Score: 0.9337
ROC AUC Score: 0.9283


In [ ]:
import pandas as pd

# Load the dataset
 # Replace with your actual dataset file

# Randomly select 10,000 rows for training
train_df = df.sample(n=10000, random_state=42)

# Select remaining 31,000 rows for testing
test_df = df.drop(train_df.index)

# Save the datasets
train_df.to_csv("training.csv", index=False)
test_df.to_csv("testing.csv", index=False)

print("Training and testing datasets saved successfully!")


Training and testing datasets saved successfully!


In [ ]:
# Step 4: Train and evaluate the AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Initialize the AdaBoost Classifier
ada = AdaBoostClassifier(random_state=42)

# Train the model
ada.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ada.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Print the evaluation metrics
print("AdaBoost Classifier Evaluation:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

AdaBoost Classifier Evaluation:
Accuracy: 0.7186
Precision: 0.6775
Recall: 0.8416
F1 Score: 0.7507
ROC AUC Score: 0.7177


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Initialize the Random Forest model
rf = RandomForestClassifier(
    random_state=42,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5
)

# K-Fold Cross Validation (Stratified for class balance)
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
import numpy as np

# Initialize Repeated Stratified K-Fold (5 folds, repeated 3 times)
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

# Perform cross-validation
cv_accuracies = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='accuracy')
cv_precisions = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='precision')
cv_recalls = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='recall')
cv_f1s = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='f1')
cv_roc_aucs = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='roc_auc')

# Print results
print("Repeated Stratified K-Fold Cross-Validation (5-Fold, 3 Repeats):")
print(f"Accuracy: {np.mean(cv_accuracies):.4f} ± {np.std(cv_accuracies):.4f}")
print(f"Precision: {np.mean(cv_precisions):.4f} ± {np.std(cv_precisions):.4f}")
print(f"Recall: {np.mean(cv_recalls):.4f} ± {np.std(cv_recalls):.4f}")
print(f"F1 Score: {np.mean(cv_f1s):.4f} ± {np.std(cv_f1s):.4f}")
print(f"ROC AUC Score: {np.mean(cv_roc_aucs):.4f} ± {np.std(cv_roc_aucs):.4f}")


# Train final model on full training data
rf.fit(X_train, y_train)

# Evaluate on test set
y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("\nFinal Model Evaluation on Test Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")


Repeated Stratified K-Fold Cross-Validation (5-Fold, 3 Repeats):
Accuracy: 0.9268 ± 0.0054
Precision: 0.8765 ± 0.0083
Recall: 0.9942 ± 0.0017
F1 Score: 0.9316 ± 0.0047
ROC AUC Score: 0.9947 ± 0.0007

Final Model Evaluation on Test Set:
Accuracy: 0.9287
Precision: 0.8786
Recall: 0.9961
F1 Score: 0.9337
ROC AUC Score: 0.9283


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load training and testing datasets
train_df = pd.read_csv("/content/retraining.csv")  # Load your training dataset
test_df = pd.read_csv("/content/retesting.csv")  # Load your testing dataset

# Separate features (X) and target (y)
X_train = train_df.drop(columns=['Result'])  # Replace 'target' with actual target column name
y_train = train_df['Result']

X_test = test_df.drop(columns=['Result'])
y_test = test_df['Result']

# Initialize the Random Forest model
rf = RandomForestClassifier(
    random_state=42,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5
)

# Initialize Repeated Stratified K-Fold (5 folds, repeated 3 times)
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

# Perform cross-validation only on training data
cv_accuracies = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='accuracy')
cv_precisions = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='precision')
cv_recalls = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='recall')
cv_f1s = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='f1')
cv_roc_aucs = cross_val_score(rf, X_train, y_train, cv=rkf, scoring='roc_auc')

# Print Cross-Validation Results
print("Repeated Stratified K-Fold Cross-Validation (5-Fold, 3 Repeats) on Training Data:")
print(f"Accuracy: {np.mean(cv_accuracies):.4f} ± {np.std(cv_accuracies):.4f}")
print(f"Precision: {np.mean(cv_precisions):.4f} ± {np.std(cv_precisions):.4f}")
print(f"Recall: {np.mean(cv_recalls):.4f} ± {np.std(cv_recalls):.4f}")
print(f"F1 Score: {np.mean(cv_f1s):.4f} ± {np.std(cv_f1s):.4f}")
print(f"ROC AUC Score: {np.mean(cv_roc_aucs):.4f} ± {np.std(cv_roc_aucs):.4f}")

# Train the final model on full training data
rf.fit(X_train, y_train)

# Evaluate on test set (kept separate)
y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("\nFinal Model Evaluation on Separate Test Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")


Repeated Stratified K-Fold Cross-Validation (5-Fold, 3 Repeats) on Training Data:
Accuracy: 0.9186 ± 0.0059
Precision: 0.8672 ± 0.0092
Recall: 0.9889 ± 0.0028
F1 Score: 0.9240 ± 0.0051
ROC AUC Score: 0.9878 ± 0.0018

Final Model Evaluation on Separate Test Set:
Accuracy: 0.9086
Precision: 0.8525
Recall: 0.9892
F1 Score: 0.9158
ROC AUC Score: 0.9082


In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Load your dataset
df = pd.read_csv('rebalanced_dataset.csv')

# Step 1: Select the features and target variable
features = [
    'Age of the patient',
    'Gender of the patient',
    'Total Bilirubin',
    'Direct Bilirubin',
    'Alkphos Alkaline Phosphotase',
    'Sgpt Alamine Aminotransferase',
    'Sgot Aspartate Aminotransferase',
    'Total Protiens',
    'ALB Albumin',
    'A/G Ratio Albumin and Globulin Ratio'
]

# Assuming 'Result' is the target column
X = df[features]
y = df['Result']

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Boosting - Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_classifier.fit(X_train, y_train)
y_pred_gb = gb_classifier.predict(X_test)
gb_accuracy = accuracy_score(y_test, y_pred_gb)
print(f'Gradient Boosting Accuracy: {gb_accuracy * 100:.2f}%')

# Step 4: Bagging - Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Accuracy: {rf_accuracy * 100:.2f}%')

# Step 5: Boosting - XGBoost Classifier
xgb_classifier = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
xgb_classifier.fit(X_train, y_train)
y_pred_xgb = xgb_classifier.predict(X_test)
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print(f'XGBoost Accuracy: {xgb_accuracy * 100:.2f}%')

# Step 6: Voting Ensemble - Combining Logistic Regression, SVC, and Decision Tree
lr = LogisticRegression(max_iter=1000)
svc = SVC(probability=True)
dt = DecisionTreeClassifier()

# Create a VotingClassifier with soft voting
voting_clf = VotingClassifier(estimators=[('lr', lr), ('svc', svc), ('dt', dt)], voting='soft')
voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)
voting_accuracy = accuracy_score(y_test, y_pred_voting)
print(f'Voting Classifier Accuracy (Soft Voting): {voting_accuracy * 100:.2f}%')

# Step 7: Voting Ensemble - Hard Voting
voting_clf_hard = VotingClassifier(estimators=[('lr', lr), ('svc', svc), ('dt', dt)], voting='hard')
voting_clf_hard.fit(X_train, y_train)
y_pred_voting_hard = voting_clf_hard.predict(X_test)
voting_accuracy_hard = accuracy_score(y_test, y_pred_voting_hard)
print(f'Voting Classifier Accuracy (Hard Voting): {voting_accuracy_hard * 100:.2f}%')

# Step 8: Compare All Models
print("\nComparison of All Models:")
print(f"Gradient Boosting Accuracy: {gb_accuracy * 100:.2f}%")
print(f"Random Forest Accuracy: {rf_accuracy * 100:.2f}%")
print(f"XGBoost Accuracy: {xgb_accuracy * 100:.2f}%")
print(f"Voting Classifier (Soft Voting) Accuracy: {voting_accuracy * 100:.2f}%")
print(f"Voting Classifier (Hard Voting) Accuracy: {voting_accuracy_hard * 100:.2f}%")


Gradient Boosting Accuracy: 87.23%
Random Forest Accuracy: 99.57%
XGBoost Accuracy: 84.79%
Voting Classifier Accuracy (Soft Voting): 97.26%
Voting Classifier Accuracy (Hard Voting): 74.31%

Comparison of All Models:
Gradient Boosting Accuracy: 87.23%
Random Forest Accuracy: 99.57%
XGBoost Accuracy: 84.79%
Voting Classifier (Soft Voting) Accuracy: 97.26%
Voting Classifier (Hard Voting) Accuracy: 74.31%


In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

# Load your dataset
df = pd.read_csv('rebalanced_dataset.csv')

# Step 1: Select the features and target variable
features = [
    'Age of the patient',
    'Gender of the patient',
    'Total Bilirubin',
    'Direct Bilirubin',
    'Alkphos Alkaline Phosphotase',
    'Sgpt Alamine Aminotransferase',
    'Sgot Aspartate Aminotransferase',
    'Total Protiens',
    'ALB Albumin',
    'A/G Ratio Albumin and Globulin Ratio'
]

# Assuming 'Result' is the target column
X = df[features]
y = df['Result']

# Step 2: Label encoding if necessary (for categorical variables)
if y.dtype == 'object':
    le = LabelEncoder()
    y = le.fit_transform(y)

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {

    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),

}

# Step 4: Evaluate models
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')  # For binary classification, use 'binary'
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')

    # Calculate AUC (Area Under the Curve)
    if len(set(y)) > 2:  # For multi-class classification
        y_prob = model.predict_proba(X_test)
        auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
    else:
        y_prob = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_prob)

    # Display results
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1-Score: {f1 * 100:.2f}%")
    print(f"AUC: {auc:.4f}")
    print("-" * 40)


Model: Random Forest
Accuracy: 99.57%
Precision: 99.29%
Recall: 99.86%
F1-Score: 99.57%
AUC: 0.9999
----------------------------------------


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
import numpy as np

# Load your dataset
df = pd.read_csv('rebalanced_dataset.csv')

# Step 1: Select the features and target variable
features = [
    'Age of the patient',
    'Gender of the patient',
    'Total Bilirubin',
    'Direct Bilirubin',
    'Alkphos Alkaline Phosphotase',
    'Sgpt Alamine Aminotransferase',
    'Sgot Aspartate Aminotransferase',
    'Total Protiens',
    'ALB Albumin',
    'A/G Ratio Albumin and Globulin Ratio'
]

# Assuming 'Result' is the target column
X = df[features]
y = df['Result']

# Step 2: Label encoding if necessary (for categorical variables)
if y.dtype == 'object':
    le = LabelEncoder()
    y = le.fit_transform(y)

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Feature Selection (Using Random Forest)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Select the features based on importance
sfm = SelectFromModel(rf, threshold=0.05)
sfm.fit(X_train, y_train)

X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)

# Step 5: Initialize models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    "XGBoost": xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVC": SVC(probability=True),
    "MLP (Neural Network)": MLPClassifier(max_iter=1000)
}

# Step 6: Hyperparameter tuning using GridSearchCV
param_grid_rf = {'n_estimators': [100, 150], 'max_depth': [5, 10, None]}
param_grid_gb = {'n_estimators': [100, 150], 'learning_rate': [0.1, 0.05]}
param_grid_xgb = {'n_estimators': [100, 150], 'learning_rate': [0.1, 0.05], 'max_depth': [3, 5]}
param_grid_lr = {'C': [0.1, 1, 10]}
param_grid_svc = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
param_grid_mlp = {'hidden_layer_sizes': [(50,), (100,), (50, 50)], 'alpha': [0.0001, 0.001]}

# GridSearchCV initialization
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=3)
grid_search_gb = GridSearchCV(GradientBoostingClassifier(), param_grid_gb, cv=3)
grid_search_xgb = GridSearchCV(xgb.XGBClassifier(), param_grid_xgb, cv=3)
grid_search_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=3)
grid_search_svc = GridSearchCV(SVC(), param_grid_svc, cv=3)
grid_search_mlp = GridSearchCV(MLPClassifier(), param_grid_mlp, cv=3)

# Step 7: Evaluate models using cross-validation
for model_name, model in models.items():
    if model_name == "Random Forest":
        grid_search_rf.fit(X_train_selected, y_train)
        best_model_rf = grid_search_rf.best_estimator_
        model = best_model_rf
    elif model_name == "Gradient Boosting":
        grid_search_gb.fit(X_train, y_train)
        best_model_gb = grid_search_gb.best_estimator_
        model = best_model_gb
    elif model_name == "XGBoost":
        grid_search_xgb.fit(X_train, y_train)
        best_model_xgb = grid_search_xgb.best_estimator_
        model = best_model_xgb
    elif model_name == "Logistic Regression":
        grid_search_lr.fit(X_train, y_train)
        best_model_lr = grid_search_lr.best_estimator_
        model = best_model_lr
    elif model_name == "SVC":
        grid_search_svc.fit(X_train, y_train)
        best_model_svc = grid_search_svc.best_estimator_
        model = best_model_svc
    elif model_name == "MLP (Neural Network)":
        grid_search_mlp.fit(X_train, y_train)
        best_model_mlp = grid_search_mlp.best_estimator_
        model = best_model_mlp

    # Step 8: Model evaluation
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')  # For binary classification, use 'binary'
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')

    # Calculate AUC (Area Under the Curve)
    if len(set(y)) > 2:  # For multi-class classification
        y_prob = model.predict_proba(X_test_selected)
        auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
    else:
        y_prob = model.predict_proba(X_test_selected)[:, 1]
        auc = roc_auc_score(y_test, y_prob)

    # Print results
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1-Score: {f1 * 100:.2f}%")
    print(f"AUC: {auc:.4f}")
    print(confusion_matrix(y_test, y_pred))
    print("-" * 50)


Model: Random Forest
Accuracy: 99.57%
Precision: 99.29%
Recall: 99.86%
F1-Score: 99.57%
AUC: 0.9999
[[2732   20]
 [   4 2787]]
--------------------------------------------------
Model: Gradient Boosting
Accuracy: 91.74%
Precision: 89.53%
Recall: 94.66%
F1-Score: 92.02%
AUC: 0.9754
[[2443  309]
 [ 149 2642]]
--------------------------------------------------
Model: XGBoost
Accuracy: 98.38%
Precision: 97.44%
Recall: 99.39%
F1-Score: 98.40%
AUC: 0.9975
[[2679   73]
 [  17 2774]]
--------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Model: Logistic Regression
Accuracy: 70.63%
Precision: 66.11%
Recall: 85.49%
F1-Score: 74.56%
AUC: 0.7507
[[1529 1223]
 [ 405 2386]]
--------------------------------------------------
